<a href="https://colab.research.google.com/github/HNR1/Bachelor-project/blob/main/fid_score_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/HNR1/pytorch-fid.git
!pip install transformers diffusers tomesd accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/HNR1/pytorch-fid.git to /tmp/pip-req-build-xd6zcuyc
  Running command git clone --filter=blob:none --quiet https://github.com/HNR1/pytorch-fid.git /tmp/pip-req-build-xd6zcuyc
  Resolved https://github.com/HNR1/pytorch-fid.git to commit 3c9ea42c486637d51a86a6e75028b47893977520
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-fid: filename=pytorch_fid-0.3.0-py3-none-any.whl size=15659 sha256=cfcab3bce2920ec5c3f5fae418e8931ddb86bdded9049ae08cdee015a7009749
  Stored in directory: /tmp/pip-ephem-wheel-cache-9p8m42in/wheels/b2/2b/ec/2e88afc5c790068df77cc5925b988a4d23258f26104c7eda95
Successfully built pytorch-fid
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 

In [ ]:
from diffusers import DiffusionPipeline
import torch, tomesd, os.path
from pytorch_fid.fid_score import calc_fid_given_lists
from time import time

In [ ]:
if os.path.exists("/content/Bachelor-project"):
  %cd Bachelors-project
  !git pull
  %cd ..
else:
  !git clone https://github.com/HNR1/Bachelor-project.git

Cloning into 'Bachelors-project'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 70 (delta 27), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (70/70), 73.93 KiB | 912.00 KiB/s, done.


In [ ]:
# parse prompts from prompts.txt
filename1 = "Bachelor-project/prompts.txt"
filename2 = "Bachelor-project/seeds_768x768.txt"
prompts = []
seeds = []
with open(filename1, 'r', encoding='UTF-8') as file:
    while line := file.readline():
        prompts.append(line.rstrip())
with open(filename2, 'r', encoding='UTF-8') as file:
    while line := file.readline():
        seeds.append(int(line.rstrip()))

In [ ]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")
pipeline.enable_attention_slicing()

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [35]:
i = 1
prompt = prompts[i]
seed = seeds[i]
image_1 = pipeline(prompt, 512, 512, generator=torch.Generator().manual_seed(seed)).images[0]
#image_2 = pipeline(prompt, 512, 512, generator=torch.Generator().manual_seed(seed)).images[0]
tomesd.apply_patch(pipeline, 0.2, use_rand=False)
image_20_1 = pipeline(prompt, 512, 512, generator=torch.Generator().manual_seed(seed)).images[0]
image_20_2 = pipeline(prompt, 512, 512, generator=torch.Generator().manual_seed(seed)).images[0]
tomesd.remove_patch(pipeline)
print('Done')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Done


In [39]:
calc_fid_given_lists(image_1, image_20_1)

3.4315978188511114

In [37]:
calc_fid_given_lists(image_1, image_20_2)

3.960233086622746

In [38]:
calc_fid_given_lists(image_20_1, image_20_2)

0.9429816562896856